# Multiclass classification

In [160]:
import sys
import os
import pandas as pd
sys.path.append('../')
import json
import random

from utils import multiclass_classification_utils

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
# Read json file

training_data_path = "training_data/CHROMA-FIT_data.json"
testing_data_path = "training_data/CHROMA-FIT_data.json"

output_file = "results/CHOMA-FIT.txt"

#------------------------------------------------------------------------------

with open(training_data_path) as f:
    training_data = json.load(f)

with open(testing_data_path) as f:
    testing_data = json.load(f)

nr_of_test_images = 64

testing_paths = []
for subject, images in testing_data.items():
    for image_name, image_data in images.items():
        if image_name != "MST_value":  # Skip the MST_value entry
            testing_paths.append(f"{subject}/{image_name}")

random.seed(42)  # Ensure reproducibility
sampled_testing_paths = random.sample(testing_paths, nr_of_test_images)

print(len(sampled_testing_paths))

# Create a set for fast lookup
sampled_testing_paths_set = set(sampled_testing_paths)

filtered_training_data = multiclass_classification_utils.remove_testing_images_from_training(training_data, sampled_testing_paths_set)
filtered_testing_data = multiclass_classification_utils.filter_testing_data(testing_data, sampled_testing_paths_set)

X_train, y_train = multiclass_classification_utils.preprocess_data(filtered_training_data)
X_test, y_test = multiclass_classification_utils.preprocess_data(filtered_testing_data)

print(len(X_train), len(X_test))

#------------------------------------------------------------------------------
# with open(training_data_path) as f:
#     training_data = json.load(f)

# with open(testing_data_path) as f:
#     testing_data = json.load(f)

# training data
# X_train, y_train = multiclass_classification_utils.preprocess_data(training_data)

# # testing data
# X_test, y_test = multiclass_classification_utils.preprocess_data(testing_data)

#------------------------------------------------------------------------------

# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#------------------------------------------------------------------------------

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


64
254 64


In [162]:

# Train-test split

# Train a Random Forest classifier
clf_LogisticRegression = LogisticRegression(multi_class='ovr', random_state=42)
clf_LogisticRegression.fit(X_train, y_train)
y_pred_clf_LogisticRegression = clf_LogisticRegression.predict(X_test)

clf_RandomForestClassifier = RandomForestClassifier(random_state=42)
clf_RandomForestClassifier.fit(X_train, y_train)
y_pred_RandomForestClassifier = clf_RandomForestClassifier.predict(X_test)

clf_SVC = SVC(kernel='rbf', random_state=42)  # You can use 'linear', 'poly', or 'rbf' as kernel types
clf_SVC.fit(X_train, y_train)
y_pred_SVC = clf_SVC.predict(X_test)




/home/dasec-notebook/Thesis/skin_tone_estimation/.conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [163]:
accuracy_log_reg = accuracy_score(y_test, y_pred_clf_LogisticRegression)
report_log_reg = classification_report(y_test, y_pred_clf_LogisticRegression)
multiclass_classification_utils.save_results(output_file, "Logistic Regression", accuracy_log_reg, report_log_reg)

# Random Forest Classifier
accuracy_rf = accuracy_score(y_test, y_pred_RandomForestClassifier)
report_rf = classification_report(y_test, y_pred_RandomForestClassifier)
multiclass_classification_utils.save_results(output_file, "Random Forest Classifier", accuracy_rf, report_rf)

# SVC
accuracy_svc = accuracy_score(y_test, y_pred_SVC)
report_svc = classification_report(y_test, y_pred_SVC)
multiclass_classification_utils.save_results(output_file, "Support Vector Classifier (SVC)", accuracy_svc, report_svc)

Results for Logistic Regression saved to results/aaaa.txt
Results for Random Forest Classifier saved to results/aaaa.txt
Results for Support Vector Classifier (SVC) saved to results/aaaa.txt


/home/dasec-notebook/Thesis/skin_tone_estimation/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dasec-notebook/Thesis/skin_tone_estimation/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dasec-notebook/Thesis/skin_tone_estimation/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th